In [2]:
using JuMP
using HiGHS
using MarketData
using Statistics
using Plots
using Random
using LinearAlgebra

In [3]:
function load_data(stocks::Array{String}, years::Int64)
    output = Array{Float64}[]
    t = Dates.now()
    period = YahooOpt(period1 = t - Year(years), period2 = t)
    for stock in stocks
        data = yahoo(stock, period)
        value = (values(data["High"]) + values(data["Low"]))/2;
        push!(output, value)
    end
    return hcat(output...)
end

function evaluate_factor(data::Matrix{Float64})
    m, n = size(data) 
    factor = zeros(m-1,n)
    mean = zeros(n)

    for i in 1:m-1
        factor[i,:] = (data[i+1,:] - (data[i,:])) ./ (data[i,:])
    end
    return factor
end

data = load_data(["PETR4.SA", "VALE3.SA", "AZUL4.SA", "BPAC11.SA", "ABEV3.SA"], 1);
factor = evaluate_factor(data);

In [10]:
m, n = size(factor)
mu = vec(mean(factor, dims=1));
Sigma = cov(factor);
sigma = sqrt.(diag(Sigma));

### Question 1

The inner primal problem can be written as:

$\begin{align*}
& \min r^Tx \\
&\sum_{i=1}^N z = \Gamma &:λ\\ 
& r_i - \mu \le z_i \sigma_i, &:π1_i \quad &\forall i = 1,\ldots,N \\
 -&(r_i - \mu) \le z_i \sigma_i, &:π2_i \quad &\forall i = 1,\ldots,N \\
& z_i \leq 1 &:β_i \quad &\forall i = 1,\ldots,N \\
& z_i \geq 0 & &\forall i = 1,\ldots,N \\
&r \in \mathbb{R}
\end{align*}$

In [26]:
function inner_primal(x, Gamma, factor)

    mu = vec(mean(factor, dims=1))
    Sigma = cov(factor)
    sigma = diag(Sigma).^(1/2)
    m, n = size(factor)

    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, 0 <= z[1:n])
    @variable(model, r[1:n])

    @constraint(model, lambda, sum(z) == Gamma)
    @constraint(model, pi1, r .- mu .<=  z .* sigma)
    @constraint(model, pi2, - (r .- mu) .<=  z .* sigma)
    @constraint(model, beta, z .≤ 1)

    @objective(model, Min, r'x)
    optimize!(model)
    return model
end

inner_primal (generic function with 1 method)

In [27]:
x = (1/n)*ones(n)
Γ = 1
model = inner_primal(x, Γ, factor);

The inner dual problem can be written as:

$\begin{align*}
& \min λΓ + \sum_{i=1}^N \beta_i + μ^T(π1 - π2) \\
& λ^T\mathbf{1} - \sigma (π1 + π2) + β \leq 0 \\ 
& (π1 - π2) = x \\
& β ≤ 0 \\
& π1 ≤ 0 \\
& π2 ≤ 0 \\
& λ \in \mathbb{R} 
\end{align*}$

In [28]:
function inner_dual(x, Γ, factor)
    mu = vec(mean(factor, dims=1))
    Sigma = cov(factor)
    sigma = diag(Sigma).^(1/2)
    m, n = size(factor)

    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, lambda)
    @variable(model, beta[1:n] <= 0)
    @variable(model, pi1[1:n] <= 0)
    @variable(model, pi2[1:n] <= 0)

    @constraint(model, z, lambda*ones(n) .- sigma .* (pi1 .+ pi2) .+ beta .<= 0)
    @constraint(model, r, pi1 .- pi2 .== x)
        
    @objective(model, Max, lambda*Γ + sum(beta) + dot(mu,(pi1 .- pi2)))
    optimize!(model)
    return model
end

inner_dual (generic function with 1 method)

In [29]:
dual_model = inner_dual(x, Γ, factor);

The optimal values must be smaller than a tolerance

In [30]:
objective_value(model) - objective_value(dual_model) < 1e-6

true

Not necessarily all optimal values will be equal, the problem can have degenerate solutions

### Question 2

In [ ]:
function first_level(Γ)
    mu = vec(mean(factor, dims=1))
    Sigma = cov(factor)
    sigma = diag(Sigma).^(1/2)
    m, n = size(factor)

    model = Model(HiGHS.Optimizer)
    set_silent(model)
    
        
    @variable(model, lambda)
    @variable(model, x[1:N] >= 0)
    @variable(model, beta[1:n] <= 0)
    @variable(model, pi1[1:n] <= 0)
    @variable(model, pi2[1:n] <= 0)

    @constraint(model, w, sum(x) == 1)
    @constraint(model, z, lambda*ones(n) .- sigma .* (pi1 .+ pi2) .+ beta .<= 0)
    @constraint(model, r, pi1 .- pi2 .== x)

    @objective(model, Max, lambda*Γ + sum(beta) + dot(mu,(pi1 .- pi2)))
    optimize!(model)
    return model
end

In [ ]:
Γs = collect(0:5)
models = first_level_problem.(Γs);

In [ ]:
obj_values = objective_value.(models);

In [ ]:
plot(Γs,obj_values,xlabel="Gamma",ylabel="Returns",legend=false)

In [ ]:
xs = Matrix(hcat([value.(model[:x]) for model in models]...))';

In [ ]:
groupedbar(xs,
        bar_position = :stack,
        bar_width=0.7,
        xticks=(1:n_stocks+1,Γs),
        legend=:outertopright,
        xlabel="Gamma",
        ylabel="Composition",
        label=permutedims(tickers))

### Questão 3

In [ ]:
Random.seed!(1234);

For the outsample returns, we sample S samples from a multivariate normal distribution with mean and covariance from the portfolio historical returns.

In [ ]:
r_dist = MvNormal(μ,Σ);

In [ ]:
N_samples = 10000
r_out_of_sample = rand(r_dist, N_samples)

Each coordinate $(i,j)$ from the following matrix $M$ means that for $\Gamma = i$, the simulation $j$ has simulated cost of $M_{ij}$

In [ ]:
return_by_gamma_scenario = xs*r_out_of_sample

Each coordinate $(i,j)$ from the following vector $p$ means that for $\Gamma = i$, the probability of the simulated portfolio return be smaller than the optimal value of the problem is $p_i$

In [ ]:
estimated_prob_portfolios = sum(return_by_gamma_scenario .< obj_values,dims=2)/N_samples

### Question 4

In [ ]:
T = 10

$\begin{align*}
& \min r^Tx \\
&\sum_{t=1}^T \theta_t = 1 &:λ\\ 
& \sum_{t=1}^T \theta_t p_t = r &:π \\
& \theta_t \leq 1 &:β \\
& \theta_t \geq 0 \\
& r \in \mathbb{R}
\end{align*}$

$p_t$ is the returns from time $t$.

In [ ]:
function inner_primal_problem(T,x)
    N = n_stocks
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variables(model,
    begin
    θ[1:T] ≥ 0
    r[1:N]
    end)

    @constraints(model,
    begin
    λ, θ .≤ 1
    π, sum(θ[t]*stock_data[n_days-T+t,:] for t in 1:T) .== r
    β, sum(θ) == 1
    end)

    @objective(model, Min, r'x)
    JuMP.optimize!(model)

    return objective_value(model), JuMP.value.(r), JuMP.value.(θ)
end

In [ ]:
inner_primal_problem(T,x)

$\begin{align*}
& \min λ + \sum_{t=1}^T \beta_t \\
& λ^T\mathbf{1} - π p_t + β^T \leq 0 \\ 
& π = x \\
& β ≤ 0 \\
& λ \in \mathbb{R} 
\end{align*}$

That is equal to 

$\begin{align*}
& \min λ + \sum_{t=1}^T \beta_t \\
& λ^T\mathbf{1} + β^T \leq πx \\
& β ≤ 0 \\
& λ \in \mathbb{R} 
\end{align*}$

In [ ]:
function inner_dual_problem(T,x)
    N = n_stocks
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variables(model,
    begin
    λ
    β[1:T] ≤ 0
    end)

    @constraints(model,
    begin
    λ*ones(T) .+ β .≤ stock_data[n_days-T+1:n_days,:]*x
    end)

    @objective(model, Max, sum(β) + λ)
    JuMP.optimize!(model)

    return objective_value(model)
end

In [ ]:
inner_dual_problem(T,x)

In [ ]:
function first_level_problem(T)
    N = n_stocks
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variables(model,
    begin
    x[1:N] ≥ 0
    λ
    β[1:T] ≤ 0
    end)

    @constraints(model,
    begin
    sum(x) == 1
    λ*ones(T) .+ β .≤ stock_data[n_days-T+1:n_days,:]*x
    end)

    @objective(model, Max, sum(β) + λ)
    JuMP.optimize!(model)

    return model
end

In [ ]:
Ts = collect(10:10:100);

In [ ]:
models = first_level_problem.(Ts);

In [ ]:
obj_values = objective_value.(models);

In [ ]:
plot(Ts,obj_values,xlabel="Past Days",ylabel="Returns",legend=false)

In [ ]:
xs = Matrix(hcat([value.(model[:x]) for model in models]...))';

In [ ]:
groupedbar(xs,
        bar_position = :stack,
        bar_width=0.7,
        xticks=(1:length(Ts),Ts),
        legend=:outertopright,
        xlabel="Past Days",
        ylabel="Composition",
        label=permutedims(tickers))